In [6]:
import numpy as np
import pandas as pd
import librosa
import sys
import glob
import os
from keras.models import load_model

In [41]:
root = 'ToiletSoundSet'
predict_folder_path = root + '/validate_data/laughing'
model_path = 'models/toilet_sound_cnn_60.h5'
class_labels = [
    "toilet_flush",
    "shower",
    "door",
    "broken_glass",
    "brushing_teeth",
    "clapping",
    "laughing",
    "door_knock",
    "children_playing"
]
target_wav_duration = 2.97
wav_data_shape_x = 128
wav_data_shape_y = 128

In [21]:
def extract_feature(file_path):
    y, sr = librosa.load(file_path, duration=target_wav_duration)
    ps = librosa.feature.melspectrogram(y=y, sr=sr)
    if ps.shape != (wav_data_shape_x, wav_data_shape_y):
        return None, None
    return ps, sr

In [42]:
model = load_model(model_path)
for root, _, files in os.walk(predict_folder_path):
    for file in files:
        file_name, file_extension = os.path.splitext(file)
        if file_extension != '.wav': continue
        file_path = os.path.join(root, file)
        ps, sr = extract_feature(file_path)
        if sr == None:
            continue
        predict_ds = []
        predict_ds.append(ps)
        predict_ds = np.array([data.reshape( (128, 128, 1) ) for data in predict_ds])
        predictions = model.predict(predict_ds)[0]
        print('============= Predict wav {} ============='.format(file_name))
        for index, predict in enumerate(predictions):
            class_name = class_labels[index]
            resultStr = '{0} {1:.2f}%'.format(class_name, predict * 100)
            print(resultStr)
        counter += 1

============= Predict wav 0002-6 =============
toilet_flush 0.00%
shower 0.00%
door 0.00%
broken_glass 0.00%
brushing_teeth 0.00%
clapping 0.09%
laughing 99.91%
door_knock 0.00%
children_playing 0.00%
============= Predict wav 0000-6 =============
toilet_flush 0.00%
shower 0.00%
door 0.00%
broken_glass 0.00%
brushing_teeth 0.00%
clapping 0.00%
laughing 100.00%
door_knock 0.00%
children_playing 0.00%
============= Predict wav 0004-6 =============
toilet_flush 6.50%
shower 0.02%
door 2.19%
broken_glass 0.58%
brushing_teeth 0.16%
clapping 3.22%
laughing 69.87%
door_knock 15.50%
children_playing 1.96%
============= Predict wav 0006-6 =============
toilet_flush 0.00%
shower 0.00%
door 0.00%
broken_glass 0.00%
brushing_teeth 0.00%
clapping 100.00%
laughing 0.00%
door_knock 0.00%
children_playing 0.00%
============= Predict wav 0001-6 =============
toilet_flush 0.00%
shower 0.00%
door 0.00%
broken_glass 0.00%
brushing_teeth 0.00%
clapping 0.00%
laughing 100.00%
door_knock 0.00%
children_playi